In [10]:
%load_ext autoreload
%autoreload 2

from mint.model.transformer import Transformer
from mint.translator import Translator, BeamSearch, GreedySearch
from mint.tokenizer import Tokenizer
from mint.dataset import Dataset
from mint.metrics import bleu, chrf2
from tqdm import tqdm
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
model = Transformer.load("../pretrained/en_sk_small")
source_tokenizer = Tokenizer.load("../tokenizers/en/")
target_tokenizer = Tokenizer.load("../tokenizers/sk/")
dataset = Dataset.load("../datasets/en_sk/")["test"]

MAX_STEPS = 10
USE_CUDA = True

device = torch.device("cuda" if torch.cuda.is_available() and USE_CUDA else "cpu")
if USE_CUDA:
    model.to(device)

def evaluate(translate, max_length=None):
    bleu_total, chrf2_total = 0, 0
    for i, sample in tqdm(enumerate(dataset), "Evaluating", total=MAX_STEPS):
        if i >= MAX_STEPS:
            break
        source = sample["source"]
        target = sample["target"]
        if max_length is not None:
            source = source_tokenizer.clip([source], max_length)[0]
            target = target_tokenizer.clip([target], max_length)[0]
        
        prediction = translate(source) 
        print(f"Source: {source}")
        print(f"Target: {target}")
        print(f"Prediction: {prediction}")
        bleu_total += bleu(prediction, target)
        chrf2_total += chrf2(prediction, target)
        
    bleu_score = bleu_total / MAX_STEPS
    chrf2_score = chrf2_total / MAX_STEPS
    print(f"BLEU: {bleu_score:.4f}, CHRF2: {chrf2_score:.4f}")
    
    return bleu_score, chrf2_score

{'n_blocks': 6, 'vocab_size': 10001, 'transformer_block_config': {'d_model': 256, 'd_feedforward': 1024, 'p_dropout': 0.1, 'attention_config': {'n_heads': 16, 'd_model': 256, 'max_seq_len': 129, 'context_window': None}}, 'embedding_config': {'vocab_size': 10001, 'd_model': 256, 'max_seq_len': 129, 'learnable_positional_embeddings': True}}


In [12]:
translator = Translator(model, source_tokenizer, target_tokenizer)
evaluate(
    lambda x: translator.translate(x, max_length=128),
    max_length=128,
)

Evaluating:  20%|██        | 2/10 [00:00<00:00, 12.90it/s]

Source: I invite you to pray for them and for those who have lost their lives.”
Target: Pozývam vás k modlitbe za nich, ako aj za tých, čo stratili život.“
Prediction: Poznávam vás, aby ste sa zaviatili a za tých, ktorí stratili ich život.“
Source: Thus God's Son came down from heaven.
Target: A vtedy Boh Syn sám zostúpil z výšky nebeskej.
Prediction: A tak aj od Boha, bol od neba v nebi.
Source: When the apostle heard what she had said he said, "Who will rid me of Marwan's daughter?"
Target: Keď apoštol počul, čo povedala, povedal, "Kto ma zbaví Marwanovej dcéry?"
Prediction: Keď je,, keď sa,, keď sa nepočul, povedal: „Kto sa obrží z marca Marwanahodeho?”


Evaluating:  60%|██████    | 6/10 [00:00<00:00, 12.73it/s]

Source: It is the fruit of the Word delivered to God’s people through His chosen shepherds.
Target: Je to ovocím Slova, ktoré je dané Božiemu ľudu prostredníctvom Jeho vyvolených pastierov.
Prediction: Je to ovocie svätého Božieho slova, podľa jeho výdesti, ktoré sa poslúchalo.
Source: The atmosphere is filled with God’s presence.
Target: Atmosféra sa začala napĺňať Božou prítomnosťou.
Prediction: Srdce je s Božou vôľou.
Source: 'Is God willing to prevent evil, but not able?
Target: Chce boh zabrániť zlu, ale nedokáže?
Prediction: A Boh je ochotný zabrániť zlatu, ale nie?
Source: With God's help, you can control the tongue.
Target: S Božou pomocou dokážeš skrotiť jazyk!
Prediction: S Božou pomocou dokážeš ovládať jazyk.
Source: Ask them," he added.
Target: Pýtajte sa ďalej,“ dodal.
Prediction: Popúšťajte ich,“ dodal.


Evaluating: 100%|██████████| 10/10 [00:00<00:00, 14.51it/s]

Source: He says, “every careless word.”
Target: On hovorí, "Každý neopatrný slovo."
Prediction: Hovorí: „Smrť sa, bez ohľadu na to,
Source: Therefore, there is no need to suppose God's existence."
Target: Preto niet dôvodu predpokladať existenciu Boha.“
Prediction: Nie je potrebné, aby sa spoznala existencie Boha."
BLEU: 0.0920, CHRF2: 0.2125


(tensor(0.0920), 0.21247061429587927)

In [13]:
translator = Translator(model, source_tokenizer, target_tokenizer, search_strategy=BeamSearch(20))

evaluate(
    lambda x: translator.translate(x, max_length=128),
    max_length=128,
)


Evaluating:  10%|█         | 1/10 [00:00<00:01,  5.88it/s]

Source: I invite you to pray for them and for those who have lost their lives.”
Target: Pozývam vás k modlitbe za nich, ako aj za tých, čo stratili život.“
Prediction: Hľadám, aby som sa modlil za nich a za tých, ktorí stratili život."
Source: Thus God's Son came down from heaven.
Target: A vtedy Boh Syn sám zostúpil z výšky nebeskej.
Prediction: Boží Syn prišiel z neba,


Evaluating:  30%|███       | 3/10 [00:00<00:01,  5.45it/s]

Source: When the apostle heard what she had said he said, "Who will rid me of Marwan's daughter?"
Target: Keď apoštol počul, čo povedala, povedal, "Kto ma zbaví Marwanovej dcéry?"
Prediction: Keď sú to, čo nepočul, povedal, že povedal: „Kto sa skryje o dcére Marwana?“


Evaluating:  40%|████      | 4/10 [00:02<00:05,  1.04it/s]

...

Evaluating:  70%|███████   | 7/10 [00:05<00:02,  1.16it/s]

Source: With God's help, you can control the tongue.
Target: S Božou pomocou dokážeš skrotiť jazyk!
Prediction: S Božou pomocou môžete ovládať jazyk.
Source: Ask them," he added.
Target: Pýtajte sa ďalej,“ dodal.
Prediction: Pýtaj sa ich,“ dodal.


Evaluating: 100%|██████████| 10/10 [00:05<00:00,  1.67it/s]

Source: He says, “every careless word.”
Target: On hovorí, "Každý neopatrný slovo."
Prediction: Hovorí: „Hľa, bez ohľadu na to,
Source: Therefore, there is no need to suppose God's existence."
Target: Preto niet dôvodu predpokladať existenciu Boha.“
Prediction: Preto nie je potrebné spoznať existenciu Boha."
BLEU: 0.0685, CHRF2: 0.2339


(tensor(0.0685), 0.2339177126671283)

In [7]:
!pip install easynmt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [14]:
from easynmt import EasyNMT
model = EasyNMT('opus-mt')

evaluate(
    lambda x: model.translate(x, source_lang='en', target_lang='sk'),
    max_length=128,
)

Evaluating:  30%|███       | 3/10 [00:02<00:04,  1.70it/s]

Source: I invite you to pray for them and for those who have lost their lives.”
Target: Pozývam vás k modlitbe za nich, ako aj za tých, čo stratili život.“
Prediction: Pozývam vás, aby ste sa modlili za nich a za tých, ktorí prišli o život.
Source: Thus God's Son came down from heaven.
Target: A vtedy Boh Syn sám zostúpil z výšky nebeskej.
Prediction: Teda Boží Syn zostúpil z neba.
Source: When the apostle heard what she had said he said, "Who will rid me of Marwan's daughter?"
Target: Keď apoštol počul, čo povedala, povedal, "Kto ma zbaví Marwanovej dcéry?"
Prediction: A keď počul apoštol, čo povedala, povedal: Kto ma zbaví dcéry Marwanovej?


Evaluating:  80%|████████  | 8/10 [00:02<00:00,  5.74it/s]

Source: It is the fruit of the Word delivered to God’s people through His chosen shepherds.
Target: Je to ovocím Slova, ktoré je dané Božiemu ľudu prostredníctvom Jeho vyvolených pastierov.
Prediction: Je to ovocie Slova dodané Bohu prostredníctvom Jeho vyvolených pastierov.
Source: The atmosphere is filled with God’s presence.
Target: Atmosféra sa začala napĺňať Božou prítomnosťou.
Prediction: Atmosféra je plná prítomnosti Boha.
Source: 'Is God willing to prevent evil, but not able?
Target: Chce boh zabrániť zlu, ale nedokáže?
Prediction: Je Boh ochotný zabrániť zlu, ale nie je schopný?
Source: With God's help, you can control the tongue.
Target: S Božou pomocou dokážeš skrotiť jazyk!
Prediction: S Božou pomocou môžeš ovládať jazyk.
Source: Ask them," he added.
Target: Pýtajte sa ďalej,“ dodal.
Prediction: Spýtaj sa ich," dodal.


Evaluating: 100%|██████████| 10/10 [00:02<00:00,  3.89it/s]

Source: He says, “every careless word.”
Target: On hovorí, "Každý neopatrný slovo."
Prediction: Hovorí: "Každé neopatrné slovo."
Source: Therefore, there is no need to suppose God's existence."
Target: Preto niet dôvodu predpokladať existenciu Boha.“
Prediction: Preto nie je potrebné predpokladať, že Boh existuje."
BLEU: 0.1696, CHRF2: 0.3911


(tensor(0.1696), 0.3910530747627739)